<a href="https://colab.research.google.com/github/NastasiaMazur/Q-A/blob/main/QA_CUAD_dataset_preparation_14.03.2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("cuad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 22450
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4182
    })
})

In [4]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  EXHIBIT 10.6

                              DISTRIBUTOR AGREEMENT

         THIS  DISTRIBUTOR  AGREEMENT (the  "Agreement")  is made by and between Electric City Corp.,  a Delaware  corporation  ("Company")  and Electric City of Illinois LLC ("Distributor") this 7th day of September, 1999.

                                    RECITALS

         A. The  Company's  Business.  The Company is  presently  engaged in the business  of selling an energy  efficiency  device,  which is  referred to as an "Energy  Saver"  which may be improved  or  otherwise  changed  from its present composition (the "Products").  The Company may engage in the business of selling other  products  or  other  devices  other  than  the  Products,  which  will be considered  Products if Distributor  exercises its options pursuant to Section 7 hereof.

         B. Representations.  As an inducement to the Company to enter into this Agreement,  the  Distributor  has  represented  that  it has or  will  have 

In [5]:
from datasets import DatasetDict
import random
from datasets import load_dataset, Dataset

# Load the original dataset
raw_datasets = load_dataset("cuad")

# Shuffle the training dataset
raw_datasets["train"].shuffle()

# Select 128 random examples for training
train_indices = random.sample(range(len(raw_datasets["train"])), 128)
train_subset = raw_datasets["train"].select(train_indices)

# Select 32 random examples for validation
valid_indices = random.sample(set(range(len(raw_datasets["train"]))) - set(train_indices), 32)
valid_subset = raw_datasets["train"].select(valid_indices)

# Create a dictionary with the small datasets

small_cuad_dataset = {
    "train": train_subset,
    "validation": valid_subset
}

small_cuad_dataset

<ipython-input-5-b2474510a7d1>:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  valid_indices = random.sample(set(range(len(raw_datasets["train"]))) - set(train_indices), 32)


{'train': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 128
 }),
 'validation': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 32
 })}

In [6]:
print("Context: ", small_cuad_dataset["train"][0]["context"])
print("Question: ", small_cuad_dataset["train"][0]["question"])
print("Answer: ", small_cuad_dataset["train"][0]["answers"])

print("Context: ", small_cuad_dataset["validation"][0]["context"])
print("Question: ", small_cuad_dataset["validation"][0]["question"])
print("Answer: ", small_cuad_dataset["validation"][0]["answers"])

Context:  Exhibit 10.1 BUSINESS DEVELOPMENT AGREEMENT THIS BUSINESS DEVELOPMENT AGREEMENT (the "Agreement") is effective as of , 2020 (the "Effective Date"), by and between LIQUIDMETAL TECHNOLOGIES, INC., a Delaware corporation having its principal place of business at 20321 Valencia Circle, Lake Forest, California 92630 ("Liquidmetal"), and EUTECTIX, LLC, a Delaware limited liability company having an address of 323 Main Street, Chatham, New Jersey 07928 ("Eutectix"). Liquidmetal and Eutectix are sometimes referred to herein individually as a "Party" or collectively as the "Parties." RECITALS WHEREAS, Liquidmetal is a global leader in the development and pursuit of applications for amorphous alloys and has developed relationships with various prominent corporations with respect to the development of parts and products made from amorphous alloys; WHEREAS, Eutectix is a global leader in the development and production of amorphous alloys and other high-quality technical alloys, and posse

During training, there is only one possible answer. We can double-check this by using the Dataset.filter() method:

In [7]:
small_cuad_dataset["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/128 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 71
})

For evaluation, however, there are several possible answers for each sample, which may be the same or different:

In [8]:
print(small_cuad_dataset["validation"][0]["answers"])
print(small_cuad_dataset["validation"][4]["answers"])

{'text': ['Access to such TSA Records shall be exercised by a Party and its Affiliates and their authorized representatives in a manner that shall not interfere unreasonably with the normal operations of the Party maintaining the TSA Records.'], 'answer_start': [40695]}
{'text': ['SUPPORT AND MAINTENANCE AGREEMENT'], 'answer_start': [161]}


In [9]:
print(raw_datasets["test"][2]["context"])
print(raw_datasets["test"][2]["question"])

Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deemed invalid. 2. The agreement and order 2.1 During the validity term of this agreement, The buyer entrust SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD or SHENZHEN LEHEYUAN TRADING CO, LTD (hereinafter refe

In [10]:
from transformers import RobertaTokenizerFast

model_checkpoint = "roberta-base"
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint)

In [11]:
tokenizer.is_fast

True

We can pass to our tokenizer the question and the context together, and it will properly insert the special tokens to form a sentence like this:

**< s> question < /s>< /s> context < /s>**

In [12]:

#print(small_cuad_dataset["train"][0]["question"])
#print(small_cuad_dataset["train"][0]["answers"])

**Achtung!**
Token indices sequence length is longer than the specified maximum sequence length for this model (8933 > 512). Running this sequence through the model will result in indexing errors

In [13]:
context = small_cuad_dataset["train"][0]["context"]
question = small_cuad_dataset["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

Token indices sequence length is longer than the specified maximum sequence length for this model (17842 > 512). Running this sequence through the model will result in indexing errors


'<s>Highlight the parts (if any) of this contract related to "Exclusivity" that should be reviewed by a lawyer. Details: Is there an exclusive dealing\xa0 commitment with the counterparty? This includes a commitment to procure all “requirements” from one party of certain technology, goods, or services or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on\xa0 collaborating or working with other parties), whether during the contract or\xa0 after the contract ends (or both).</s></s>Exhibit 10.1 BUSINESS DEVELOPMENT AGREEMENT THIS BUSINESS DEVELOPMENT AGREEMENT (the "Agreement") is effective as of, 2020 (the "Effective Date"), by and between LIQUIDMETAL TECHNOLOGIES, INC., a Delaware corporation having its principal place of business at 20321 Valencia Circle, Lake Forest, California 92630 ("Liquidmetal"), and EUTECTIX, LLC, a Delaware limited liability company having an address of 323 Main Street, Chatham, New Jersey 07928 ("Eutectix")

In [14]:
inputs = tokenizer(
    question,
    context,
    max_length=512, # I changed from 100 to 512
    truncation="only_second", # to truncate the context (which is in the second position) when the question with its context is too long
    stride=50, # to set the number of overlapping tokens between two successive chunks (here 50)
    return_overflowing_tokens=True, # to let the tokenizer know we want the overflowing tokens
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))



<s>Highlight the parts (if any) of this contract related to "Exclusivity" that should be reviewed by a lawyer. Details: Is there an exclusive dealing  commitment with the counterparty? This includes a commitment to procure all “requirements” from one party of certain technology, goods, or services or a prohibition on licensing or selling technology, goods or services to third parties, or a prohibition on  collaborating or working with other parties), whether during the contract or  after the contract ends (or both).</s></s>Exhibit 10.1 BUSINESS DEVELOPMENT AGREEMENT THIS BUSINESS DEVELOPMENT AGREEMENT (the "Agreement") is effective as of, 2020 (the "Effective Date"), by and between LIQUIDMETAL TECHNOLOGIES, INC., a Delaware corporation having its principal place of business at 20321 Valencia Circle, Lake Forest, California 92630 ("Liquidmetal"), and EUTECTIX, LLC, a Delaware limited liability company having an address of 323 Main Street, Chatham, New Jersey 07928 ("Eutectix"). Liquidme

As we can see, our example has been in **split into 22 inputs**, each of them containing the question and some part of the context. (TRY CtrlF "..Highlight the parts")

 the question (“Bernadette Soubirous”) only appears in the 7th inputs, so by dealing with long contexts in this way we will create some training examples where the answer is not included in the context. For those examples, the labels will be start_position = end_position = 0 <s>(so we predict the [CLS] token).</s> We will also set those labels in the unfortunate case where the answer has been truncated so that we only have the start (or end) of it. For the examples where the answer is fully in the context, the labels will be the index of the token where the answer starts and the index of the token where the answer ends.

 The dataset provides us with the start character of the answer in the context, and by adding the length of the answer, we can find the end character in the context. To map those to token indices, we will need to use the offset mappings we studied in Chapter 6. We can have our tokenizer return these by passing along return_offsets_mapping=True:




In [15]:
inputs = tokenizer(
    question,
    context,
    max_length=512,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

As we can see, we get back the usual input IDs, token type IDs, and attention mask, as well as the offset mapping we required and an extra key, overflow_to_sample_mapping. The corresponding value will be of use to us when we tokenize several texts at the same time (which we should do to benefit from the fact that our tokenizer is backed by Rust). Since one sample can give several features, it maps each feature to the example it originated from. Because here we only tokenized one example, we get a list of 0s:

In [16]:
inputs["overflow_to_sample_mapping"] # we get 22

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

But if we tokenize more examples, this will become more useful:

In [17]:
inputs = tokenizer(
    small_cuad_dataset["train"][2:6]["question"],
    small_cuad_dataset["train"][2:6]["context"],
    max_length=512, # I changed from 100 to 512
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 135 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3].


As we can see, <s>the first three examples (at indices 2, 3, and 4 in the training set) each gave 55 features and the last example (at index 5 in the training set) gave 7 features.</s>

at indices 2 gave 99 features, at indices 3 gave 51 features, at indices 4 gave 10 features, at indices 4 gave 27 features.

This information will be useful to map each feature we get to its corresponding label. As mentioned earlier, those labels are:

*   (0, 0) if the answer is not in the corresponding span of the context
*   (start_position, end_position) if the answer is in the corresponding span of the context, with start_position being the index of the token (in the input IDs) at the start of the answer and end_position being the index of the token (in the input IDs) where the answer ends

To determine which of these is the case and, if relevant, the positions of the tokens, we first find the indices that start and end the context in the input IDs. We could use the token type IDs to do this, but since those do not necessarily exist for all models (DistilBERT does not require them, for instance), we’ll instead use the **sequence_ids()** method of the BatchEncoding our tokenizer returns.

Once we have those token indices, we look at the corresponding offsets, which are tuples of two integers representing the span of characters inside the original context. We can thus detect if the chunk of the context in this feature starts after the answer or ends before the answer begins (in which case the label is (0, 0)). If that’s not the case, we loop to find the first and last token of the answer:



In [34]:
answers = small_cuad_dataset["train"][1,2,4,7,8]["answers"] #2:6 --: 1,2,4,7,8 SEE the PROBLEM down
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  145,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  405,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


Answer:  {'text': ['DISTRIBUTOR AGREEMENT'], 'answer_start': [44]}

**PROBLEM:**

In [35]:
#print("Answer: ", small_cuad_dataset["train"][0]["answers"]) #Nothing 0, 3, 5,6,9 ok:1,2,4,7,8
print("Answer: ", small_cuad_dataset["train"][7]["answers"])

Answer:  {'text': ['In the event of termination, neither party, their heirs nor successors shall issue any challenge whatsoever to contest the termination.'], 'answer_start': [4073]}
